### Analyzing amino acid distributions

This notebook assumes that you have initialized the workspace in the workspace/workspace_initialization example and aligned the data in the alignment/alignment.ipynb notebook.

In [ ]:
import os
from scipy import stats
import pandas

from pepars.analysis import statistics as virus_stats
from pepars.analysis import amino_acids as AA_analysis
from pepars.plotting import plotting
from pepars.plotting import DNA as DNA_plotting
plotting.init_notebook_mode()

from protfarm.analysis.Analysis_Set import Analysis_Set
from protfarm.workspace import Database as db
from protfarm.workspace import Workspace as ws
from protfarm.analysis.Sequence_Library import Sequence_Library

In [ ]:
# The data path represents the location of all protein engineering sequencing experiments
DATA_PATH = os.path.join("..", "example_data")

# Each experiment is given its own name and subdirectory in the DATA PATH
# An experiment is a group of samples, all using the same variant region
EXPERIMENT_NAME = "demo"

# Choose the sample name to visualize
SAMPLE_NAME = "sample_1"

# Remove sequences below this count threshold
COUNT_THRESHOLD = 1

In [ ]:
ws.set_data_path(DATA_PATH)
ws.set_experiment(EXPERIMENT_NAME)
ws.set_active_alignment(db.get_alignments()[0])
sample = db.get_library(SAMPLE_NAME)
template = db.get_template_by_id(ws.get_active_alignment().library_templates[sample.id])

In [ ]:
# Get the sequence counts for the requested sample
sequence_reads = Sequence_Library(sample)
sequence_counts = sequence_reads.get_sequence_counts(by_amino_acid=True, count_threshold=COUNT_THRESHOLD, filter_invalid=True)

In [ ]:
# Plot the amino acid biases as a log2 fold change over the exptected value, given this sample's template
DNA_plotting.plot_amino_acid_bias(
    sequence_counts,
    template.get_variant_template(),
    sample_name=sample.name,
    interactive=True)

In [ ]:
# An alternative way to look at these distributions is based on their z-score. First we convert to Pandas
sequence_counts = pandas.DataFrame.from_dict(sequence_counts, orient="index")

# Now we get the number of sequences of each amino acid at each position (this ignores counts)
amino_acid_counts_by_position = AA_analysis.get_amino_acid_counts_by_position(sequence_counts.index)

# We get the expected biases, based on our template
amino_acid_biases = AA_analysis.get_amino_acid_codon_biases([template.get_variant_template()])

In [ ]:
# Calculate the Z score, based on the given test type
p_values, z_scores = virus_stats.get_significance_of_amino_acid_ratios(
    amino_acid_counts_by_position,
    amino_acid_biases,
    multiple_comparison_correction=True,
    test_type=virus_stats.Test_Type.STANDARDIZATION)

In [ ]:
# Plot the z scores
plotting.plot_significance_z_scores(z_scores, interactive=True)